In [ ]:
'''
%%writefile ns.py
start = time.time()
end = time.time()
print(end-start)
'''
import tensorflow as tf
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import os as os
import sys
from pathlib import Path

mhsdir = Path(os.getcwd()).parent
sys.path.append(os.path.join(mhsdir, 'src'))

import ns

datarawdir = os.path.join(mhsdir, 'data\\raw\\YC')
dataprepdir = os.path.join(mhsdir, 'data\\preprocessed')
dataprepnsdir = os.path.join(mhsdir, 'data\\preprocessed\\ns')

tempdatarawdir = os.path.join(mhsdir, 'temp\\data\\raw\\YC')
tempdataprepdir = os.path.join(mhsdir, 'temp\\data\\preprocessed')
tempdataprepnsdir = os.path.join(mhsdir, 'temp\\data\\preprocessed\\ns')

# Concatenate all data into one file

In [ ]:
df = pd.read_csv(os.path.join(dataprepnsdir,'SA_Chile_2786_ns.csv'))

In [ ]:
for _ in range(100):
    df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df.to_csv(os.path.join(dataprepnsdir, 'Shuffle_Total_294089.csv'), index = False)

In [ ]:
ActTerms = ['D1', 'M01', 'M03', 'M04', 'M06', 'M07', 'M09', 
            'Y01', 'Y02', 'Y03', 'Y04', 'Y05', 'Y06', 'Y07', 'Y08', 'Y09', 'Y10', 'Y12', 'Y15', 'Y20', 'Y30']

@tf.function
def transform_to_yc(x):
    terms = tf.constant([[0.033333, 1.0, 3.0, 4.0, 6.0, 7.0, 9.0,
                          12.0, 24.0, 36.0, 48.0, 60.0, 72.0, 84.0, 96.0, 108.0, 120.0, 144.0, 180.0, 240.0, 360.0]],dtype = tf.float32)   
    batch = tf.shape(x)[0]
    sz = tf.constant(terms.shape[1])
    val1 = tf.divide(terms, tf.slice(x, [0,0], [batch,1]))
    val2 = tf.math.exp(-val1)
    val3 = tf.divide(1.0 - val2, val1)
    y = tf.add(tf.concat([tf.ones([batch,sz,1]), tf.zeros([batch,sz,1]), tf.expand_dims(-val2, 2)], axis = -1),
               tf.math.multiply(tf.expand_dims(val3, 2), [[-1.0, 1.0, 1.0]]))
    return  tf.einsum('...ik,...k->...i',y, tf.slice(x, [0 , 1],[batch, 3]))


df = pd.DataFrame(columns = ActTerms)
for filename in os.listdir(dataprepnsdir):
    filepath = os.path.join(dataprepnsdir, filename)
    df1 = pd.read_csv(filepath)
    df1 = df1[df1['Thau'].notnull()]
    nsm = df1[['Thau','Alpha0', 'Alpha1', 'Alpha2']].astype('float32').to_numpy()
    termm = transform_to_yc(nsm).numpy()
    sharedCol = [col for col in ActTerms if col in df1.columns]
    df2 = pd.DataFrame(data = -256.0 * np.ones([df1.shape[0], len(ActTerms)]), columns = ActTerms, dtype = np.float32)
    df2[sharedCol] = df1[sharedCol].to_numpy()
    df2 = df2.mask(df2 == -256.0, termm)
    df = df.append(df2)